In [4]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = dset.MNIST('/home/daiict/Desktop/udit/C-GAN./data' ,train=True, download= True,
                       transform = transform)
test_set = dset.MNIST('/home/daiict/Desktop/udit/C-GAN./data' ,train=False, download=True,
                       transform = transform)

In [6]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64,shuffle=True)
print len(train_loader), len(test_loader)

938 157


In [7]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 0
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

In [4]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
p_fingerprints[(p_fingerprints==0)] = -1

c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

#Normalise the features
c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

fingerprints = np.concatenate((p_fingerprints,c_fingerprints),axis=1)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = fingerprints.shape
labels = labels[1:]

In [5]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)
total_pos = np.sum(labels2)
print("tot_positive",total_pos)


('tot_positive', 48)


In [6]:
ind = np.arange(no_examples)
np.random.shuffle(ind)
fingerprints = fingerprints[ind]
labels2 = labels2[ind]

In [7]:
print fingerprints.shape

(3423, 144)


In [8]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        p_train_data = fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_train_data[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(float)
    else:
        None
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [9]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = fingerprints[s_ind:e_ind]
    else:
        None
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  
        

In [10]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,250)
        self.l4 = nn.Linear(250,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [12]:
min_fn = 15
max_fp = 160
cm_list = []
batch_size = 128
for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(20,50,20)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(3000):
            train_batch,target = get_train_batch(batch_size,binary = True,validation_iter = val_iter)
            model_op = mymlp(train_batch)
            #print(model_op.type)
            #print(target.type)
            loss = criterion(model_op,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter,binary = True)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        if(fn < 3):
            cm_list.append(wcf)
            if(fn<4):
                if(fp < max_fp):
                    max_fp = fp
                    model_path = os.getcwd() + '/model_all_feat_file' + fname
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([662,   7,  11,   4]))
('tn, fp, fn, tp: ', array([661,   8,  11,   4]))
('tn, fp, fn, tp: ', array([658,  11,  12,   3]))
('tn, fp, fn, tp: ', array([664,   5,  11,   4]))
('tn, fp, fn, tp: ', array([665,   4,  11,   4]))
('tn, fp, fn, tp: ', array([660,   9,  12,   3]))
('tn, fp, fn, tp: ', array([662,   7,  10,   5]))
('tn, fp, fn, tp: ', array([663,   6,  11,   4]))
('tn, fp, fn, tp: ', array([661,   8,  11,   4]))
('tn, fp, fn, tp: ', array([663,   6,  11,   4]))
('tn, fp, fn, tp: ', array([660,   9,  10,   5]))
('tn, fp, fn, tp: ', array([661,   8,  11,   4]))
('tn, fp, fn, tp: ', array([661,   8,  12,   3]))
('tn, fp, fn, tp: ', array([663,   6,  12,   3]))
('tn, fp, fn, tp: ', array([661,   8,  12,   3]))
('tn, fp, fn, tp: ', array([661,   8,  11,   4]))
('tn, fp, fn, tp: ', array([662,   7,  11,   4]))
('tn, fp, fn, tp: ', array([662,   7,  11,   4]))
('tn, fp, fn, tp: ', array([662,   7,  12,   3]))
('tn, fp, fn, tp: ', array([657,

In [ ]:
os.getcwd() + '/model_all' + fname